# Supercompressible: create dabase

L. F. Pereira (lfpereira@fe.up.pt)\
September 10, 2020


This notebook creates the **design of experiments** and the metadata required to run the **numerical simulations**.

**Note**: the functions to create the parametric `Abaqus` scripts were developed without taking `f3dasm` into consideration, in order to show the adaptability of the framework.

Let's import the required libraries.

In [1]:
# imports

# standard library
import os
import pickle
from collections import OrderedDict

# third-party
import numpy as np
from SALib.sample import sobol_sequence
import pandas as pd

# local library
from f3dasm.run.utils import create_main_file
from f3dasm.abaqus.run.utils import run_sims

In [2]:
seed = 1

Now, we need to define the problem.

Let's start with the folder name and the number of points.

In [3]:
example_name = 'example_2'

n = 2  # number of points

In [4]:
import shutil
try:
    shutil.rmtree(example_name)
except:
    pass

Then, let's define the **variables of the problem**, both the variables that are defined by the design of experiments scheme and the fixed variables (i.e. the variables that are common for all simulations).

**Note**: the use of an ```OrderedDict``` is not mandatory, but forces variable order in the ```Pandas``` frame.

In [5]:
doe_variables = OrderedDict({'phi': [0, 90],
                            'psi': [0, 90],
                            'D': [2, 10],
                            'W_D': [3, 8]})

The inputs have to be transformed, because the design of experiments uses variables that differ from the variables required by the model.

In [6]:
# just demonstrative
def transform_inputs(inputs):
    
    new_inputs = {var_name: variable for var_name, variable in inputs.items() if var_name not in['phi', 'psi', 'D', 'W_D']}
    phi = inputs['phi']
    psi = inputs['psi']
    new_inputs['layup'] =  [phi, -phi, psi, -psi]
    new_inputs['notch_radius'] = inputs['D']
    new_inputs['width'] = inputs['W_D']*inputs['D']

    return new_inputs

Another important step is the definition of the information related with the numerical simulations.

In [7]:
sim_info = {'abstract_model': 'library_UDG_INEGI.models.OH.OH_FEM',
            'sim_info': {'OH': {'model_name': 'OH',
                                'test_case': 'tension',
                                'laminate_material_system': 'IM7_8552',
                                'ply_thickness': .125,
                                'job_info': {'name': 'OH',
                                             'description': '',
                                             'userSubroutine': 'library_UDG_INEGI.subroutines.vumat_CDM_q3d_CZMfric_2019.for'}}},
            'transform_inputs': transform_inputs}


The simulation definition is quite straightforward.

First, we need to define the `abstract_model`. That is, the class that is initialized given the variables values and contains all the methods required to successfully create, run and post-process the numerical model.

Second, we need to define the specific parameters of each simulation. In this example, there's a first simulation that performs a linear buckling analysis and another that performs a riks analysis. The second simulation depends on the results of the first. The use of an `OrderedDict` is of great importance to guarantee the right order of the simulations.

Finally, we define also a transformation of inputs. This is simply a mapping from the variables used in the design of experiments to the variables required to create the model.

Now, using a proper design of experiments scheme, we can create the design of experiments. Here, we simply use Sobol sequence implementation from ```Salib``` library.

In [8]:
dim = len(doe_variables)
points_sobol = sobol_sequence.sample(n, dim)
for i, lims in enumerate(doe_variables.values()):
    points_sobol[:, i] = points_sobol[:, i] * (lims[1] - lims[0]) + lims[0]

Finally, we store the information in a dictionary and save it. Note the design of experiments is stored in a ```Pandas.DataFrame```. This is a convenient way for the subsequent application of Machine Learning.

In [9]:
doe_variables_ls = list(doe_variables.keys())
points = pd.DataFrame(points_sobol, columns=doe_variables_ls)
data = {'doe_variables': doe_variables,
        'points': points,
        'sim_info': sim_info,}

In [10]:
create_main_file(example_name, doe_variables, points, sim_info)

## Run simulations

First, let's open the created file and see what it contains.

In [11]:
with open(os.path.join(example_name, 'DoE.pkl'), 'rb') as file:
    data = pickle.load(file)
    
print(data.keys())

dict_keys(['doe_variables', 'points', 'sim_info', 'run_info', 'version'])


In [12]:
run_sims(example_name, n_sims=1, abaqus_path='abaqus',
         keep_odb=True, dump_py_objs=False, gui=True)